In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random
from scipy.signal import find_peaks
from scipy.ndimage import uniform_filter1d
import copy
from sklearn.decomposition import PCA

from sklearn.mixture import GaussianMixture
from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.neighbors import NearestNeighbors

from scipy import stats

### Generate Recording
- Create a recording from a single trace of specified time duration (Poisson spike distribution)

In [2]:
def gen_recording(templates, dur, rate, dt):
    
    dt = dt / 1000 # Convert to ms
    timesteps = int(dur/dt)
    spike_prob = rate * dt
    
    spike_train = np.random.uniform(size=(timesteps, 1)) < spike_prob
    spike_idxs = np.where(spike_train == True)[0]
    
    template_dur = np.shape(templates)[-1]
    
    # Add all spikes into recording
    num_elecs = np.shape(templates)[-2]
    num_fields = np.shape(templates)[0]
    
    recording = np.zeros((num_fields, num_elecs, timesteps))
    
    for i in range(len(spike_idxs)):
        spike_idx = spike_idxs[i]
        if spike_idx + template_dur < timesteps:
            recording[:, :, spike_idx:spike_idx+template_dur] = templates
    
    return recording

### Generate All Recordings

In [3]:
def gen_all_recordings(fields, num_temps, dur, rate, dt):

    temp_indices = np.random.choice(len(fields[0]), num_temps, replace=False)

    recordings = []
    for idx in temp_indices:
        recording = gen_recording(fields[:, idx], dur, rate, dt)
        recordings.append(recording)

    recordings = np.array(recordings)
    
    return recordings

### Load Data

In [17]:
# Get all templates from files

folder_name = 'ziad_mearec_templates/'
mea_name = '100MEA75'

#file_name = 'mag_templates_flattened_x15_cell2_rotate_n50_100MEA75.npy'
file_name = f'mag_templates_flattened_5-25bound_0-2cells_n300_{mea_name}.npy'
#file_name = 'mag_templates_flattened_noise_cells2-3_ylim36.0_zlim[-535.5, 535.5]_Neuropixels-128.npy'
#file_name = 'mag_templates_flattened_noise_cells2-3_xlim[60, 100]_ylim[60, 100]_zlim[-535.5, 535.5]_Neuropixels-128.npy'

#file_name = 'mag_templates_flattened_5-25bound_2-4cells_n300_Neuropixels-128.npy'
#file_name = 'mag_templates_flattened_5-25bound_2-4cells_n300_Neuropixels-128.npy'
#file_name = f'mag_templates_flattened_5-25bound_0-2cells_n300_Ziad_Custom_-2_100MEA75_Ex.npy'

with open(folder_name + file_name, 'rb') as f:
    v = np.load(f)[:, :, :]
    bx = np.load(f)[:, :, :]
    by = np.load(f)[:, :, :]
    bz = np.load(f)[:, :, :]
    locs = np.load(f)
    rots = np.load(f)

addl_files = []
addl_files = [f'mag_templates_flattened_5-25bound_2-4cells_n300_{mea_name}.npy',
              f'mag_templates_flattened_5-25bound_4-6cells_n300_{mea_name}.npy']

# addl_files = ['mag_templates_flattened_5-25bound_2-4cells_n300_Ziad_Custom_-2_100MEA75_Ex.npy',
#               'mag_templates_flattened_5-25bound_4-6cells_n300_Ziad_Custom_-2_100MEA75_Ex.npy']

for file in addl_files:
    with open(folder_name + file, 'rb') as f:
        v2 = np.load(f)[:, :, :]
        bx2 = np.load(f)[:, :, :]
        by2 = np.load(f)[:, :, :]
        bz2 = np.load(f)[:, :, :]
        locs2 = np.load(f)
        rots2 = np.load(f) 

        v = np.concatenate((v, v2), axis=0)
        bx = np.concatenate((bx, bx2), axis=0)
        by = np.concatenate((by, by2), axis=0)
        bz = np.concatenate((bz, bz2), axis=0)
        locs = np.concatenate((locs, locs2), axis=0)
        rots = np.concatenate((rots, rots2), axis=0)

fields = np.array([v, bx, by, bz])
print("V: ", np.shape(v))
print("Bx: ", np.shape(bx))
print("By: ", np.shape(by))
print("Bz: ", np.shape(bz))
print("Locations: ", np.shape(locs))
print("Rotations: ", np.shape(rots))

V:  (1800, 100, 224)
Bx:  (1800, 100, 224)
By:  (1800, 100, 224)
Bz:  (1800, 100, 224)
Locations:  (1800, 3)
Rotations:  (1800, 3)


### Condition Number Analysis
- all_temps: Array of number of cells (transmitters) to include (randomly sample) in analysis
- n_per_cell: Number of templates per cell (fixed)
- fields_to_run: Maximum value of 7 - 0=V, 1=Bx, 2=By, 3=Bz, 4=BxByBz, 5=ByBz, 6=VBx; so value of 3 runs V, Bx, and By
- cell_indices: Which cells to run (cells 2 and 3 are the best for B-fields)
- generate_recordings: If it is enabled, the H matrix is populated with recordings; otherwise, it uses templates
- dur: Duration of simulated recording
- rate: Poisson average spiking rate for recording generation
- dt: Timestep of simulation (fixed)

- Outputs:
    - Prints condition number for each iteration of each field
    - Averages condition numbers together at the end of the sim

In [19]:
#all_temps = [20, 40, 60, 80, 100, 120, 140, 160, 180, 200]
all_temps = [50, 113]
n_per_cell = 300
fields_to_run = 7
field_labels = ['V', 'Bx', 'By', 'Bz', 'BxByBz', 'ByBz', 'VBx']

all_s = []

generate_recordings = False
dur = 1        # Duration (s) of fake recording             
rate = 10      # Spiking rate (Hz) for Poisson process
dt = 0.03125   # dt of simulation

verbose = False

for num_temps in all_temps:
    cell_indices = [2, 3]
    #cell_indices = [0, 1, 2, 3, 4, 5]

    iters = 30

    # Store all condition numbers
    conds = np.zeros((fields_to_run, iters))
    
    # Store all singular values
    s_vals = np.zeros((fields_to_run, iters, num_temps))
    
    # Create H matrix from flattened recordings for each field
    for j in range(iters):
        if verbose:
            print("Iteration:", j)
        
        # Get fields corresponding to specified cells
        num_fields = np.shape(fields)[0]
        total_num_templates = len(cell_indices)*n_per_cell
        num_elecs = np.shape(fields)[2]
        num_tsteps = np.shape(fields)[3]

        # Create array of only templates from specified cells (all field types)
        sub_fields = np.zeros((num_fields, total_num_templates, num_elecs, num_tsteps))
        for n, cell_idx in enumerate(cell_indices):
            sub_fields[:, n*n_per_cell:(n+1)*n_per_cell] = fields[:, cell_idx*n_per_cell:(cell_idx+1)*n_per_cell]

        # Select num_temps random templates and store in templates
        idxs = np.random.choice(len(sub_fields[0]), num_temps, replace=False)
        templates = sub_fields[:, idxs]
#         if verbose:
#             print(np.shape(templates))
        
        if generate_recordings == True:
            templates = gen_all_recordings(templates, num_temps, dur, rate, dt)
            templates = np.transpose(templates, (1, 0, 2, 3))
            print("Templates new shape: ", np.shape(templates))
            
        # Flatten electrode data - H has shape (num_temps x (timesteps * num_electrodes))
        for i in range(fields_to_run):
            
            # Run BxByBz
            if i == 4:
                H = np.transpose(templates[1:4], (1, 0, 2, 3))
                H = H.reshape((len(templates[0]), -1))
            # Run ByBz
            elif i == 5:
                H = np.transpose(templates[2:4], (1, 0, 2, 3))
                H = H.reshape((len(templates[0]), -1))
            # Run VBx
            elif i == 6:
                H = np.transpose(templates[0:2], (1, 0, 2, 3))
                #print("H shape 1: ", np.shape(H))
                H[:, 0] = H[:, 0] / np.linalg.norm(H[:, 0])
                H[:, 1] = H[:, 1] / np.linalg.norm(H[:, 1])
                H = H.reshape((len(templates[0]), -1))
            # Run single field
            else:
                H = templates[i].reshape((len(templates[0]), -1))
            
            if verbose:
                print("H shape:", np.shape(H))
            
            H = np.matrix(H / np.linalg.norm(H))
            cov = H * H.getH()
            #cov = np.cov(H)
            
            u, s, vh = np.linalg.svd(cov)
            s = np.sqrt(s)
            cond = s[0]/s[-1]
            if verbose:
                print(field_labels[i], ":", cond)

            conds[i, j] = cond
            s_vals[i, j] = s
        if verbose:
            print("")

    means = np.mean(conds, axis=1)
    stds = np.std(conds, axis=1)
    if verbose == True:
        print("Field \t Mean Condition Number \t Std")
        for q in range(len(means)):
            print(field_labels[q], '\t', means[q], '\t', stds[q])

        print()
    for q in range(len(means)):
        print(means[q], '\t', stds[q], '\t', str(iters), '\t', end="")
    print()
    all_s.append(s_vals)

13.396849540863267 	 3.7352446306534888 	 30 	7.479887417154441 	 1.3637339353478646 	 30 	18.463356544707942 	 3.7937236286470997 	 30 	23.358386465399104 	 6.425513378508477 	 30 	6.654426272606913 	 1.5491729587420522 	 30 	10.841289644957554 	 2.1331979301569053 	 30 	5.561197175887508 	 1.0168028141139849 	 30 	
30.649278671030068 	 6.897604594410327 	 30 	14.444843127074767 	 2.7922469473639637 	 30 	34.5380880682776 	 9.153186068590394 	 30 	40.636261377579 	 8.183418397427845 	 30 	10.143908945232887 	 1.8573923051988108 	 30 	17.97027002535226 	 3.6030096090871155 	 30 	10.394892871668693 	 2.0408124824835143 	 30 	


In [19]:
a = np.random.random((20, 100, 300))

b = a*1.0
c = a*1.0
b[:, :, 150:] = 0
c[:, :, :150] = 0

a = a.reshape((20, -1))
b = b.reshape((20, -1))
c = c.reshape((20, -1))

aH = np.dot(a, a.T)
qH = np.dot(b, b.T) + np.dot(c, c.T)

print(np.sum(aH-qH))

-5.275069270282984e-11


In [20]:
int(dur/dt*1000)

32000

### Singular Value Analysis

In [ ]:
mean_s = np.mean(all_s[0][0], axis=0)
plt.plot(np.log10(mean_s))
mean_s = np.mean(all_s[0][1], axis=0)
plt.plot(np.log10(mean_s))
mean_s = np.mean(all_s[0][2], axis=0)
plt.plot(np.log10(mean_s))
mean_s = np.mean(all_s[0][3], axis=0)
plt.plot(np.log10(mean_s))

plt.legend(['V', 'Bx', 'By', 'Bz'])

In [ ]:
idx = 3
vals = np.log10(all_s[0][idx])
means = np.mean(vals, axis=0)
stds = np.std(vals, axis=0)

for i in range(len(means)):
    print(means[i], '\t', stds[i], '\t', str(30))

In [ ]:
np.shape()

In [ ]:
H = np.random.random((10, 30))
H = np.matrix(H / np.linalg.norm(H))
cov = H * H.getH()
#cov = np.cov(H)
u, s, vh = np.linalg.svd(H)
u, sCov, vh = np.linalg.svd(cov)

print("Singular Values H Matrix: ", s, '\n')
print("Singular Values Covariance Matrix: ", sCov, '\n')
print("Sqrt Singular Values Covariance Matrix: ", np.sqrt(sCov), '\n')


In [22]:
a = np.random.random((10, 20))
a = np.matrix(a)

In [23]:
u, s, vh = np.linalg.svd(a)
print(s)

[6.96490076 1.95017604 1.71541091 1.58259562 1.34856072 1.24987526
 0.96036467 0.81450536 0.72654359 0.48972336]


In [24]:
cov = a * a.getH()

In [26]:
u, s, vh = np.linalg.svd(cov)
print(s)
print(np.sqrt(s))

[48.50984257  3.80318657  2.94263458  2.5046089   1.81861603  1.56218816
  0.92230031  0.66341898  0.52786558  0.23982897]
[6.96490076 1.95017604 1.71541091 1.58259562 1.34856072 1.24987526
 0.96036467 0.81450536 0.72654359 0.48972336]


In [27]:
a2 = a - np.mean(a)
cov = a2 * a2.getH()

In [28]:
u, s, vh = np.linalg.svd(cov)
print(s)
print(np.sqrt(s))

[3.95533684 3.14185803 2.73311264 2.3142591  1.7662542  1.32644173
 0.91103641 0.66629506 0.36424103 0.18069747]
[1.98880287 1.77252871 1.65321282 1.52126891 1.32900497 1.15171252
 0.95448227 0.81626899 0.60352385 0.42508525]
